In [3]:
# Import pinceone key from environment
import os

PINECONE_KEY = os.environ.get('PINECONE_KEY')
print(PINECONE_KEY)

None


In [2]:
print(PINECONE_KEY)

None


In [2]:
import pinecone
# Initialize Pinecone
pinecone.init(api_key=PINECONE_KEY, environment='gcp-starter')
# Get index recipe-index
index = pinecone.Index("recipe-index")
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Load pre-trained model tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.to(device)  # Move model to GPU if available
model.eval()  # Set model to evaluation mode

def get_embeddings(text):
    # Tokenize input text
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0).to(device)
    # Get the embeddings
    with torch.no_grad():
        outputs = model(input_ids)
        embeddings = outputs[0][0, 1:-1, :]
    # Get the average embedding
    return embeddings.mean(dim=0).cpu().numpy().tolist()

# Query it with a vector
vector = get_embeddings("Chocolate cake with frosting")
pinecone_results = index.query(vector=vector, top_k=5, include_distance=True, include_vector=True, include_metadata=True)

In [7]:
pinecone_results

{'matches': [{'id': '148221',
              'metadata': {'all_text': 'Name: Chocolate Glaze (That Hardens '
                                       'when Cool) Description: ISO: Chocolate '
                                       'Glaze that dries hard using cocoa and '
                                       'without... Saw this and found the '
                                       'answer.  This would be good now with '
                                       "Valentine's Day is near.  Try using it "
                                       'like those chocolate dips they have in '
                                       'ice cream shops; it hardens into a '
                                       'shell on the ice cream.  Or use it to '
                                       'top your favorite cookie, cupcake, '
                                       'cheesecake, brownie, etc.. '
                                       'Ingredients: 2 margarine, 2 hot water, '
                              

In [4]:
for idx, score, metadata in zip(pinecone_results.ids[0], pinecone_results.scores[0], pinecone_results.metadata[0]):
    print(f"ID: {idx}, Score: {score}")
    print("Associated Text Data:", metadata.get("all_text"))
    print("---" * 10)

TypeError: 'NoneType' object is not subscriptable